# `Part 2/3: Trade data collection with Binance API`

The purpose of this series of notebooks is to showcase real-time candle, trade and order book data collection methods from Binance exchange.

Data is collected through websockets and then stored in SQL databases.

Possible applications of the stored data include:
* creation of custom alerts for discretionary trading 
* development of a fully automated systematic training system based on a set of predetermined rules

#### Author: Vladislav Semin

## 1. Import libraries

In [1]:
import time
import calendar
from datetime import datetime

import pandas as pd #to create a dataframe

import websocket
from binance.client import Client # Import the Binance Client
from binance.websockets import BinanceSocketManager # Import the Binance Socket Manager

import sqlalchemy as db
from sqlalchemy import create_engine

## 2. Setup Binance websocket connection

**IMPORTANT NOTE:** 
This kernel will not run without API keys. Register an account at Binance crypto exchange website to get your API keys:
* https://www.binance.com/en/register

In [2]:
# Input Binance API keys
# PUBLIC = ''
# SECRET = ''

In [3]:
# Setup Binance websocket connection
client = Client(api_key=PUBLIC, api_secret=SECRET)
bm = BinanceSocketManager(client)

## 3. Select cryptocurrency pair

In [4]:
# We will be collecting data for the following cryptocurrency pair
pair = 'ETHBTC'

## 4. Setup SQL database

In [5]:
# Name of SQL database
trade_db = 'binance_trades_{}'.format(pair)

In [6]:
# Set up engine to append the values to the SQL database
engine = create_engine('sqlite:///{}.db'.format(trade_db), echo=False)

## 5. Set up data collection format

In [7]:
# handle_message is a callback function requested by .start_trade_socket method of BinanceSocketManager class
def handle_message(msg):
    '''
    # import global dataframe into which we will save our data
    global t_df
    '''
    global engine
    global trade_db
    
    if msg['e'] == 'error':    
        print(msg['m'])
        
    else:
        # To calculate the amount of currency exchanged in bitcoins we convert strings to floats.
        btc_exchanged = float(msg['p']) * float(msg['q'])
        
        # Timestamp converted to date time with microseconds
        time = msg['T'] / 1000
        time = datetime.fromtimestamp(time).strftime("%Y-%m-%d %H:%M:%S.%f") 
        
        # Buy or sell?
        # If "m" is true, buyer is the market maker, then someone sold at OB sell price 
        if msg['m'] == True:
            event_side = 'SELL'
        else:
            event_side = 'BUY'
            
        # Streaming trades data
        stream_dict = {"Trade Time": time, "Trade t-stamp": msg['T'], "Event t-stamp": msg['E'],
                       "Pair": msg['s'], "BUY/SELL": event_side,
                       "Price": msg['p'], "Quantity": msg['q'], "BTC value": btc_exchanged,  
                       "Buyer order ID": msg['b'], "Seller order ID": msg['a'], "Trade ID": msg['t'], 
                       'Buyer MM': msg['m'], "Event": msg['e'], 'Ignore': msg['M']}
        
        stream_df = pd.DataFrame([stream_dict]).set_index('Trade Time')  
        
        '''
        # Updating global dataframe
        t_df = t_df.append(stream_df, sort=False)
        '''
        
        # real-time export of streaming dataframes to SQLite database        
        stream_df.to_sql(trade_db, if_exists="append", con=engine)
        
        # Event message acknowledging new trades occurring on the exchange
        print('### {} trade data updated at {}'.format(msg['s'], time))

## 6. Collect trade data

In [8]:
# Connect to collect data
conn_key_trade = bm.start_trade_socket(pair, handle_message)

# Start data collection
bm.start()

# For testing purposes, streaming period is set to 15 seconds. Max connection time is 24 hrs.
time.sleep(15) 

bm.stop_socket(conn_key_trade)

bm.close()

### ETHBTC trade data updated at 2020-05-07 00:36:36.393000
### ETHBTC trade data updated at 2020-05-07 00:36:37.755000
### ETHBTC trade data updated at 2020-05-07 00:36:37.755000
### ETHBTC trade data updated at 2020-05-07 00:36:39.123000
### ETHBTC trade data updated at 2020-05-07 00:36:39.697000
### ETHBTC trade data updated at 2020-05-07 00:36:39.697000
### ETHBTC trade data updated at 2020-05-07 00:36:40.222000
### ETHBTC trade data updated at 2020-05-07 00:36:40.222000
### ETHBTC trade data updated at 2020-05-07 00:36:44.806000
### ETHBTC trade data updated at 2020-05-07 00:36:44.806000
### ETHBTC trade data updated at 2020-05-07 00:36:44.806000
### ETHBTC trade data updated at 2020-05-07 00:36:44.806000
### ETHBTC trade data updated at 2020-05-07 00:36:46.288000
### ETHBTC trade data updated at 2020-05-07 00:36:46.288000
### ETHBTC trade data updated at 2020-05-07 00:36:46.493000
### ETHBTC trade data updated at 2020-05-07 00:36:46.493000
### ETHBTC trade data updated at 2020-05

## 7. Check the data appended to SQL database

In [9]:
# Select and show data from the dataframe
binance_trades = engine.execute("SELECT * FROM " + trade_db).fetchall()

# We can see raw SQL data here: 
binance_trades

[('2020-05-07 00:36:36.393000', 1588800996393, 1588800996395, 'ETHBTC', 'BUY', '0.02224200', '0.84100000', 0.018705522, 710218637, 710218636, 174576575, 0, 'trade', 1),
 ('2020-05-07 00:36:37.755000', 1588800997755, 1588800997756, 'ETHBTC', 'SELL', '0.02224000', '0.00800000', 0.00017792, 710218631, 710218651, 174576576, 1, 'trade', 1),
 ('2020-05-07 00:36:37.755000', 1588800997755, 1588800997756, 'ETHBTC', 'SELL', '0.02223800', '0.21600000', 0.004803408, 710218649, 710218651, 174576577, 1, 'trade', 1),
 ('2020-05-07 00:36:39.123000', 1588800999123, 1588800999125, 'ETHBTC', 'BUY', '0.02224000', '1.43600000', 0.031936639999999995, 710218668, 710218657, 174576578, 0, 'trade', 1),
 ('2020-05-07 00:36:39.697000', 1588800999697, 1588800999698, 'ETHBTC', 'SELL', '0.02223800', '0.50700000', 0.011274666000000001, 710218661, 710218672, 174576579, 1, 'trade', 1),
 ('2020-05-07 00:36:39.697000', 1588800999697, 1588800999698, 'ETHBTC', 'SELL', '0.02223700', '0.16700000', 0.0037135790000000003, 7102

In [10]:
# Or we can transform SQL data into pandas dataframe
df = pd.DataFrame(binance_trades, columns=["Trade Time", "Trade t-stamp", "Event t-stamp", "Pair", "BUY/SELL",
                                           "Price", "Quantity", "BTC value", "Buyer order ID", "Seller order ID", "Trade ID", 
                                           'Buyer MM', "Event", "Ignore"]).set_index("Trade Time")

df.head()

,Trade t-stamp,Event t-stamp,Pair,BUY/SELL,Price,Quantity,BTC value,Buyer order ID,Seller order ID,Trade ID,Buyer MM,Event,Ignore
Trade Time,,,,,,,,,,,,,
2020-05-07 00:36:36.393000,1588800996393,1588800996395,ETHBTC,BUY,0.02224200,0.84100000,0.018706,710218637,710218636,174576575,0,trade,1
2020-05-07 00:36:37.755000,1588800997755,1588800997756,ETHBTC,SELL,0.02224000,0.00800000,0.000178,710218631,710218651,174576576,1,trade,1
2020-05-07 00:36:37.755000,1588800997755,1588800997756,ETHBTC,SELL,0.02223800,0.21600000,0.004803,710218649,710218651,174576577,1,trade,1
2020-05-07 00:36:39.123000,1588800999123,1588800999125,ETHBTC,BUY,0.02224000,1.43600000,0.031937,710218668,710218657,174576578,0,trade,1
2020-05-07 00:36:39.697000,1588800999697,1588800999698,ETHBTC,SELL,0.02223800,0.50700000,0.011275,710218661,710218672,174576579,1,trade,1


## References:
#### Binance API Python libraries used in this demonstration:

https://gist.github.com/alexbrillant/961502146a7fc5d03205f9b07b8535f5 - Binance Socket Manager class and its methods

https://github.com/binance-exchange/python-binance